# 프로토콜(Protocol)
## 객체의 설계도
## 프로토콜의 활용

## optional

### 프로토콜의 다양한 활용법 중에서 마지막으로, 프로토콜에서 사용되는 optional 키워드에 대해 알아보자
- #### 프로토콜을 구현할 때는 기본적으로 프로토콜의 명세에 포함된 모든 프로퍼티와 메소드, 그리고 초기화 구문을 구현해야 한다
- #### 그렇지 않으면 필요한 항목의 구현이 누락되었다는 오류가 발생한다

### 하지만 구현하는 객체에 따라 특별히 필요하지 않는 프로퍼티나 메소드, 초기화 구문이 있을 수 있다
- #### 이런 메소드까지 모두 일일이 구현해야 한다면 상당히 번거로워진다

### 이런 상황을 방지하기 위한 해법이 바로 선택적 요청(Optional Requirement)이라고 불리는 문법이다
- #### 이 문법은 프로토콜에서 선언된 프로퍼티나 메소드, 초기화 구문 등 프로토콜을 구현할 때 작성해야 하는 요소들을 필수 사항에서 선택 사항으로 바꾸어 준다

### 프로토콜을 정의할 때 선택적 요청을 개별 요소마다 지정할 수 있는데, 
- #### 이때 optional 키워드를 사용하여 프로퍼티나 메소드, 초기화 구문 앞에 표시한다
    - #### 이 키워드가 붙은 요소들은 프로토콜을 구현할 때 반드시 구현하지 않아도 된다는 것을 의미한다

### 프로토콜에서 optional 키워드를 사용하려면 약간의 제약이 있다
- #### 프로토콜 앞에 @objc를 표시해야 한다

### @objc는 파운데이션 프레임워크에 정의된 어노테이션의 일종으로서,
- #### 이 어노테이션이 붙은 코드나 객체를 Objective-C 코드에서도 참조할 수 있도록 노출됨을 의미한다
- 실제로 자신이 정의한 프로토콜이 Objective-C 코드와 상호 동잘할 일이 없도라도 말이다
- #### 또한, @objc 어노테이션이 붙은 프로토콜은 구조체나 열거형 등에서 구현할 수 없다. 오로지 클래스만 이 프로토콜을 구현할 수 있다

### 정리
- #### optional 키워드가 붙은 선택적 요청 프로토콜은 클래스만 구현할 수 있다는 뜻이다
- #### 이런 의미에서 optional 키워드 역시 클래스 전용 프로토콜임을 뜻하는 것이라고 할 수 있다

### MsgDelegate 프로토콜을 정의하는 예제

```
import Foundation

@objc
protocol MsgDelegate {
    @objc optional func onReceive(new:Int)
}
```

- #### @objc 어노테이션이 파운데이션 프레임워크에서 정의되어 있으므로 import Foundation 구문을 통해 파운데이션 프레임워크를 참조할 수 있도록 해 주어야 한다
- #### 이 프로토콜에는 onReceive(new:)라는 메소드가 정의되어 있는데, optional 키워드가 추가되어 있으므로 반드시 구현하지 않아도 된다
    - #### 이 메소드는 새로운 메시지의 개수를 델리게이트로 할당된 객체에 알려주는 역활을 한다

### 실제로 메시지를 받고 처리하는 MsgCenter 역활을 하는 클래스

```
class MsgCenter {
    var delegate: MsgDelegate?
    var newMsg: Int = 0
    
    func msgCheck() {
        if newMsg > 0 { // 새로운 메시지가 도착했다면
            self.delegate?.onReceive?(new: self.newMsg)
            self.newMsg = 0
        }
    }
}
```

- #### 델리게이션 구현과 유사한 구조이다
- #### msgCheck( ) 메소드가 호출되면 새로운 메시지가 있는지 없는지를 검사해서 있을 때는 델리게이트로 할당된 객체의 onReceive(new:) 메소드를 호출한 다음, 새로운 메시지의 개수를 0으로 설정
    - #### 이때 optional 키워드가 붙은 메소드를 호출할 때는 옵셔널 체인처럼 사용하면 된다
        - #### 다만 이때는 메소드의 결과값이 옵셔널이 아니라 메소드 자체가 옵셔널이므로 괄호 사이에 ? 연산자를 작성해야 한다
        ```
        .onReceive?(new:newMsg)
        ```

### 여기에서 메소드가 반환하는 값이 일반 값이라 할지라도 옵셔널 메소드 형식으로 사용하면 결과값도 옵셔널 타입임을 주의할 필요가 있다
- #### 옵셔널 메소드는 옵셔널 체인과 같은 방식으로 동작하므로 만약 델리게이트 클래스에서 onReceive(new:) 메소드를 구현했다면 구문은 실행되고, 델리게이트 클래스의 onReceive(new:) 메소드가 호출된다
- #### 만약 해당 메소드를 클래스에서 반드시 구현했다는 확신이 있으면 옵셔널 연산자 대신 강제 해제 연산자를 사용할 수도 있다
- #### 이때 메소드의 결과값은 일반 값이 반환된다. (메소드의 호출 결과가 성공이면)

```
.onReceive!(new:self.newMsg)
```

### MsgDelegate 프로토콜을 구현하고 있는 Watch 클래스를 보자

```
class Watch: MsgDelegate {
    var msgCenter: MsgCenter?
    
    init(msgCenter: MsgCenter) {
        self.msgCenter = msgCenter
    }
    
    func onReceive(new: Int) {
        print("\(new) 건의 메시지가 도착했습니다")
    }
}
```

- #### 이 클래스에서는 onReceive(new:) 메소드를 구현하여 새로운 메시지의 개수를 출력한다
- #### onReceive(new:)메소드는 프로토콜에서 optional 키워드로 선언되어 있으므로 필수로 구현하지 않아도 된다
    - #### 새로운 메시지에 대한 도착 알림이 필요할 때만 구현하면 된다
    - #### 구현하지 않아도 별다른 문제는 없다.
    - #### 단지 새로운 메시지 도착에 대한 알림을 못받는 것 뿐이다.

### 실제 사용하는 코코아 터치 프레임워크에서는 프로토콜마다 정의해야 할 메소드가 상당히 많다
- #### 매우 디테일하게 동작하는 앱을 만들어야 한다면 이들 메소드 대부분이 의미있는 역활을 하겠지만, 메소드 중 일부만을 사용해야 할 경우라면 모든 메소드를 구현해야 하는 것은 상당한 부담이다

### 그래서 해당 프로토콜에서 반드시 필요한 메소드들만을 제외하고 나머지는 다음 예시와 같이 대부분 optional 키워드로 선언되어 선택적으로 구현할 수 있도록 제공하고 있다

### 프로토콜의 선택적 구현의 예

```
public protocol UIImagePickerControllerDelegate : NSObjectProtocol {
    
    @available(iOS 2.0, *)
    optional public func imagePickerController(_ picker: UIImagePickerController, didFinishPickerController)
    
    @available(iOS 2.0, *)
    optional public func imagePickerControllerDidCancel(_ picker: UIImagePickerController)
}
```

### 마무리
- #### 처음에는 프로토콜이 왜 필요한지 의문이 생길 수 있지만, 막상 앱을 만들면서 코드를 구현하다 보면 프로토콜을 사용함으로써 많은 기능을 구현 할 수 있다는 사실을 깨닫게 된다
- #### 이들 기능의 대부분은 개발자들을 편하게 만들어주고 코드의 작성량을 줄여주는 것이기도 하다.
- #### 지금 당장은 프로토콜의 사용 목적이 와 닿지 않더라도 실제 앱을 구현해 가면서 프로토콜의 중요성에 대해 알아가도록 하자